In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src/')

In [2]:
from package.logger import Timed, rlog, setup
from package import storage

setup("INFO")

In [3]:
import os

import folium
import geopandas as gpd
import pandas as pd

from package import strtime
from package.osm import osm
from package.mcr.data import NetworkType, OSMData
from package.mcr5.h3 import (
    get_h3_cells_for_nodes,
    get_h3_cells_for_bbox,
    plot_h3_cells_on_folium,
)
from package.mcr5.mcr5 import MCR5
from package import storage

from package.geometa import GeoMeta
from package.mcr.data import NetworkType

In [4]:
geo_meta_path = "../data/geometa.pkl"
city_id = "Koeln"
stops = "../data/cleaned/stops.csv"
structs = "../data/structs.pkl"
bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"

In [5]:
geo_meta = GeoMeta.load(geo_meta_path)
# geo_data = OSMData(
# 	geo_meta,
# 	city_id,
# 	additional_network_types=[NetworkType.DRIVING, NetworkType.CYCLING],
# )
# nodes = geo_data.osm_nodes
# edges = geo_data.osm_edges
# driving_nodes, _, _ = geo_data.additional_networks[NetworkType.DRIVING]

In [6]:
lm_data = storage.read_any_dict("../data/location_mappings.pkl")
location_mappings = lm_data["location_mappings"]
car_location_mappings = lm_data["car_location_mappings"]

In [7]:
from command.step_config import (
    get_bicycle_public_transport_config,
    get_car_only_config,
    get_bicycle_only_config,
    get_public_transport_only_config,
    get_walking_only_config,
)

In [8]:
configs = {}


def get_bicyle_public_transport_config_ready():
    initial_steps, repeating_steps = get_bicycle_public_transport_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        bicycle_price_function="next_bike_no_tariff",
        bicycle_location_path=bicycle_location_path,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
    }


def get_car_only_config_ready():
    initial_steps, repeating_steps = get_car_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": car_location_mappings,
        "max_transfers": 1,
    }


def get_bicycle_only_config_ready():
    initial_steps, repeating_steps = get_bicycle_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        bicycle_price_function="next_bike_no_tariff",
        bicycle_location_path=bicycle_location_path,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
    }


def get_public_transport_only_config_ready():
    initial_steps, repeating_steps = get_public_transport_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
        structs_path=structs,
        stops_path=stops,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 2,
    }


def get_walking_only_config_ready():
    initial_steps, repeating_steps = get_walking_only_config(
        geo_meta_path=geo_meta_path,
        city_id=city_id,
    )
    return {
        "init_kwargs": {
            "initial_steps": initial_steps,
            "repeating_steps": repeating_steps,
        },
        "location_mappings": location_mappings,
        "max_transfers": 0,
    }


configs["bicycle_public_transport"] = get_bicyle_public_transport_config_ready
configs["car"] = get_car_only_config_ready
configs["bicycle"] = get_bicycle_only_config_ready
configs["public_transport"] = get_public_transport_only_config_ready
configs["walking"] = get_walking_only_config_ready

In [9]:
mcr5_output_path = "../data/mcr5/Koeln"

In [10]:
if os.path.exists(mcr5_output_path):
	raise Exception("Output path already exists")

In [11]:
for key, config in configs.items():
    rlog.info(f"Running MCR5 for {key}")

    with Timed.info("Loading config"):
        config = config()
        mcr5 = MCR5(**config["init_kwargs"])

    output_path = os.path.join(mcr5_output_path, key)

    location_mappings = config["location_mappings"]

    with Timed.info("Running MCR5"):
        errors = mcr5.run(
            location_mappings,
            start_time="08:00:00",
            output_dir=output_path,
            max_transfers=config["max_transfers"],
        )

[19:11:52] INFO     Running MCR5 for bicycle_public_transport                                       ]8;id=233862;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=481299;file:///tmp/ipykernel_5966/1908360647.py#2\2]8;;\

           INFO     Loading config                                                                  ]8;id=112411;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=568200;file:///tmp/ipykernel_5966/1908360647.py#4\4]8;;\

           INFO     Using existing OSM data                                                         ]8;id=546837;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=393462;file:///tmp/ipykernel_5966/1908360647.py#4\4]8;;\

           INFO     Loading OSM network from cache (walking)                                        ]8;id=302593;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=848718;file:///tmp/ipykernel_5966/1908360647.py#4\4]8;;\

[19:12:52] INFO     Using existing OSM data                                                         ]8;id=112756;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=589271;file:///tmp/ipykernel_5966/1908360647.py#4\4]8;;\

...

           INFO     Running MCR5                                                                   ]8;id=813664;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=444565;file:///tmp/ipykernel_5966/1908360647.py#12\12]8;;\

Starting:   0%|          | 0/1782 [00:00<?, ?it/s]

[19:52:16] INFO     All processes finished.                                                        ]8;id=649917;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=119796;file:///tmp/ipykernel_5966/1908360647.py#12\12]8;;\

           WARNING  32 errors occurred during the analysis.                                        ]8;id=278139;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=440050;file:///tmp/ipykernel_5966/1908360647.py#12\12]8;;\

           INFO     Running MCR5 done (4:53 minutes)                                               ]8;id=904801;file:///tmp/ipykernel_5966/1908360647.py\1908360647.py]8;;\:]8;id=733429;file:///tmp/ipykernel_5966/1908360647.py#12\12]8;;\

In [12]:
import pickle

errors = storage.read_any_dict("../data/mcr5/Koeln/bicycle_public_transport/errors.pkl")
print(errors[0]["logs"])

IndexError: list index out of range

**small area**  
2s with limits  
20s without limits

**full area**  
2s with limits  
9m58s without limits
